<a href="https://colab.research.google.com/github/arizonaCameraLab/computationalImaging/blob/master/coherence/ComputationalPhotographyHomework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational Photography
### David J. Brady
### University of Arizona

### Homework 1
### Srping 2024





## Homework 1

Homework 1 consists of 4 problems exploring
1. Coherence functions and spectroscopy
2. Mutual coherence of incoherent sources
3. Rotational shear interometry and
4. Spatial resolution

## Libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d, convolve
from skimage import color, data, restoration, exposure
from skimage import io
from google.colab import files
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams.update({'font.size': 16})

## Question 1. Coherence functions and spectroscopy

*   Section 6.2 of the text defines coherence functions of the optical field. Describe these functions in your own words. Explain why it is necessary to use these functions and discuss any assumptions related to their definitions.
*   Consider an LED light source emitting light with a center wavelength of 600 nm. The spectrum is distributed according to a Gaussian distribution with standard variation 3 nm. Plot the spectrum of this source and the its mutual coherence function $\Gamma (\tau)$. Make sure to use physical units (e.g. time for $\Gamma (\tau)$) along the horizontal axes of your graphs. Estimate the coherence time (e.g. the value of $\tau$ for which $|\Gamma (\tau )|$ falls to half its maximum) for this source.
*  Measuring with a Michelson interferometer, what path delay $\delta$ corresponds to this coherence time?
* If the LED emits 1 milliwatt of optical power and you observe it at a range of 10 meters using an interferometer with a 1 cm aperture, estimate the number of photons you might measure in a 1 second exposure and the the time bandwidth product for the field over your measurement


## Question 2. Mutual coherence of incoherent sources

Text equation 6.22 describes the cross spectral density of a a Fresnel zone incoherent source as
$$
W(\Delta x,\Delta y, q, \nu )={l^2\over \lambda^2z^2}
e^{-i{2\pi\nu q\over c z}}
\int\int
 S(x,y,\nu )e^{i{2\pi\nu \over c z}(x\Delta x+y\Delta y )}
dxdy
$$
Neglecting constants, the magnitude of the cross spectral density is
$$
|W(\Delta x,\Delta y, q, \nu )|=\left |
\int\int
 S(x,y,\nu )e^{i{2\pi\nu \over c z}(x\Delta x+y\Delta y )}
dxdy\right |
$$

Suppose that $S(x,y,\nu )= f(x,y) e^{-\frac{(\nu - \nu_o)^2}{\sigma^2}}$, where $\nu_0=500$ THz and $\sigma= 5$ THz. $f(x,y)$ is a spatial pattern showing the number 3 such that the number is approximately 3 mm high.


*   What is the center wavelength of the spectral distribution? What is the width of the spectral in nm? What color is this?
*   For this system, plot $\left |W(\Delta x,\Delta y, q, \nu )\right |$ at a range of 1 meter and at a range of 10 meters, with plot axes in appropriate spatial units.
*  Estimate the coherence cross section at a range of 1 meter and at a range of 10 meters.
*  Discuss and explain your results.





## Question 3. Rotational shear interferometry


*   Figure 6.6 in the text shows experimental rotational shear interferometer data, but physical units are missing on the plots. Try to replicate this image using a model for the spectrum with Gaussian peaks representing each of the peaks in the left spectral traces. Assume that the spectral peaks for the uper trace are at 450, 525 and 625 nm. For the lower spectrum assume 500 and 600 nm. Assume that the sources are 2 meters from the RSI. Deterimine the angular placement of the sources needed to create the RSI image and the pixel pitch (in microns) on the RSI.
*   Create a model image of the RSI pattern for observing the source of problem 2 at a range of 20 cm.



## Question 4. Spatial Resolution



*   Estimate the transverse and longitudinal resolution of a confocal microscope with a 1 mm aperture and a 1 mm working distance.
*   Estimate the transverse and longitudinal resolution of a camera with 1 cm apeture imaging objects at a range of 1 meter.
*   Diffraction limited optical resolution depends only on the aperture extent, not on physical processing. Explain why.
*   Discuss algorithmic and coding strategies for increasing resolution.
